# Monitor - Digital Referral & HIVST Dashboard 1

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Monitor---Digital-Referral-&amp;-HIVST-Dashboard-1" data-toc-modified-id="Monitor---Digital-Referral-&amp;-HIVST-Dashboard-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Monitor - Digital Referral &amp; HIVST Dashboard 1</a></span></li></ul></div>

In [1]:
%run "../../xUtilz/utilz_only_loader.ipynb"

<IPython.core.display.Javascript object>

In [2]:
db = pd.read_csv( "../../datasets/LG/IN/181112_Referral_Reasons.csv" )

In [3]:
db["CHV Name"] = "CHV " +  pd.Series( pd.factorize( db["CHV Name"])[0] + 1).astype( str)
db["Patient Name"] = "Patient " +  pd.Series( pd.factorize( db["Patient Name"])[0] + 1).astype( str)
db.drop( "CHV Phone", axis=1, inplace=True)

In [4]:
reasons_groupz = { 
    "Neonates" : [ "months", "umbilical", "umbllical"], 
    "Pregnancy" : ["delivery"], 
    "U5 Danger Signs" : [ "Danger", "Diarrhoea", "Diarrhea", "Dairrhea", "Coughing", "Cough", "Fever","Pneumonia", "Pnuemonia",
                        "Pheumonia", ],
    "HIVST" : ["reactive", "partner", "intimate"], 
    "Unsupported Cases" : []
}

db["Category Referral Reason"] = db["Referral Reason"].apply( lambda x: searchString(x, reasons_groupz) ) 

var_bucket_reasons =  "Category Referral Reason"
var_all_reasons = "All"
var_Unsupported = "Unsupported Cases"
var_HIVST = "HIVST"
var_Display_Colz = ["Date of referral", "CHV Name", "Patient Name" ,"Referral Reason", "Reason for Referral"]

In [5]:
# 1. set Date of referral to a datetime type 
db["Date of referral"] = pd.to_datetime( db["Date of referral"], format="%d-%b-%y") # inplace=True)

# 2. Extract Month and year categories 
db["Year"] = db["Date of referral"].dt.year 
db["Month"] = db["Date of referral"].dt.strftime('%b-%y') 
db["Day_of_Week"] = db["Date of referral"].dt.weekday_name.str[:3] 

db.sort_values( by='Date of referral', inplace=True)

LAST_UPDATED = db["Date of referral"].max().strftime( '%d-%b-%Y')

#db.head().T

In [6]:
display(HTML('<h2>Last Updated: %s</h2>'%LAST_UPDATED))

In [7]:

options_list = [ var_all_reasons] + db[var_bucket_reasons].unique().tolist()
#print( options_list )

### plot them
def plot_bar_reasons(df, t):
    ax = sns.countplot( data=df, x=var_bucket_reasons , order=df[var_bucket_reasons].value_counts().index) 
    # set % 
    total = len( df)
    for p in ax.patches:
        height = p.get_height()
        if not( pd.isnull(height) ):
            ax.text(p.get_x()+p.get_width()/2.,
                height + 1,
                '{:1.0f} %'.format(height/total*100),
                ha="center") 

    ax.set_title( t )
    return ax


def plot_bar_timing(df, t):
    ax = sns.countplot( data=df, x="Day_of_Week", order=["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"] )
    # set % 
    total = len( df )
    for p in ax.patches:
        height = p.get_height()
        if not( pd.isnull(height) ): 
            ax.text(p.get_x()+p.get_width()/2.,
                height + 0.1,
                '{:1.0f} %'.format(height/total*100),
                ha="center") 

    ax.set_title( t ) ; 
    return ax

def plot_word_cloud( df, t ):
    stop_words = STOPWORDS.union( ["CHV", "referral" ] )
    ## Translating Swahili to Eng 
    plt.imshow( WordCloud( 
        background_color="white", 
        max_font_size=50, 
        max_words=200, 
        stopwords=stop_words,
    ).generate( " ".join( df["Referral Reason"].fillna('No_Input') ) ) , 
               interpolation='bilinear'  )
    plt.axis( "off" )
    plt.title( t )
    
def plot_hivst_reasons(df, t):
    ax = sns.countplot( data=df, x="Referral Reason" ) 
    # set % 
    total = len( df )
    for p in ax.patches:
        height = p.get_height()
        if not( pd.isnull(height) ):        
            ax.text(p.get_x()+p.get_width()/2.,
                height + 0.1,
                '{:1.0f} %'.format(height/total*100),
                ha="center") 

    ax.set_title( t ) ;  
    return ax

def plot_timeline( df, t):
    myFmt = DateFormatter("%d-%b") 
    
    #ax = sns.countplot( data=df, x="Date of referral" ) 
    ax = df['Date of referral'].value_counts().sort_index().plot.line()
    ax.set_title( t )  
    ax.xaxis_date( )
    ax.xaxis.set_minor_locator(WeekdayLocator(byweekday=MO)) ## intended 
    ax.xaxis.set_major_locator( WeekdayLocator(byweekday=MO, interval=1) ) ## only label at every 4th week 
    #ax.xaxis.set_major_formatter( DateFormatter('%H:%M:') )
    ax.xaxis.set_major_formatter( myFmt)
    return ax


@interact( group_by=options_list )
def plot_by( group_by): 
    if( group_by == var_all_reasons):
        df = db
    else:
        df = db[ db[var_bucket_reasons] == group_by ] 
    
    title = '%s Reasons - %s'% (group_by, LAST_UPDATED )
    title_all = '%s Reasons - %s'% (var_all_reasons, LAST_UPDATED )
    title_unsupported = '%s Reasons - %s'% (var_Unsupported, LAST_UPDATED )
    title_HIVST = '%s Reasons - %s'% (var_HIVST, LAST_UPDATED )
    title_timing = 'Timing: %s Referrals - %s' % (group_by, LAST_UPDATED )

    #f, a = plt.subplots( nrows=2, ncols=3, figsize=(20,15) )

    plt.figure(figsize=(20,15))
    grid = plt.GridSpec( 3, 3, wspace=0.4, hspace=0.3)

    plt.subplot( grid[0,0] )
    plot_bar_reasons(db, title_all )

    plt.subplot( grid[0,1] )
    plot_bar_timing( df, title_timing)

    plt.subplot( grid[0,2] )
    plot_hivst_reasons( db[ db[var_bucket_reasons] == var_HIVST], title_HIVST )

    ## unsupported cases only
    plt.subplot( grid[1,0] )
    plot_word_cloud( db[ (db[var_bucket_reasons]!= var_HIVST) & (db[var_bucket_reasons] == var_Unsupported)] , title_unsupported)

    ## word cloud all cases - no HIVST by default b/c not user entered 
    plt.subplot( grid[1,1] )
    plot_word_cloud( df if group_by == var_HIVST else df[df[var_bucket_reasons]!=var_HIVST], "Referral Reasons - "+group_by)
    #plot_word_cloud(db, 'Referral Reasons')

    ## timeline 
    plt.subplot( grid[2,0:] )
    plot_timeline( df, title_all)

    plt.tight_layout() 

    
    


A Jupyter Widget

In [8]:
#db[ db[var_bucket_reasons] == var_HIVST]